<a href="https://colab.research.google.com/github/achett/mlflow/blob/main/MLFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install mlflow
!pip install statsmodels
!pip install numpy
!pip install pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.2/20.2 MB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.6/147.6 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.2/80.2 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.9/202.9 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 kB 3.9 MB/s eta 0:00:00


In [3]:
import os
import mlflow
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error

In [4]:
import urllib
print("Password/Enpoint IP for localtunnel is:",urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))

Password/Enpoint IP for localtunnel is: 34.125.171.41


In [5]:
%%writefile app.py
import os
import mlflow
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error
from getpass import getpass

os.environ['MLFLOW_TRACKING_USERNAME'] = 'achett2'
os.environ['MLFLOW_TRACKING_PASSWORD'] = '2af8bb9bfaeec663848b89f850e29831f075854c'

mlflow.set_tracking_uri('https://dagshub.com/achett2/my-first-repo.mlflow')

# Load the dataset
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/airline-passengers.csv"
data = pd.read_csv(url, usecols=[1], engine='python')
data = data.values.flatten()  # Convert dataframe to 1D array

# Split data into train and test sets
split = int(len(data) * 0.80)
train, test = data[0:split], data[split:len(data)]

# Set the MLflow experiment name
mlflow.set_experiment('Time Series ARIMA with statsmodels')

# Start an MLflow run
with mlflow.start_run():
    # Log dataset statistics
    mlflow.log_param("Dataset size", len(data))
    mlflow.log_param("Training size", len(train))
    mlflow.log_param("Test size", len(test))

    # Fit an ARIMA model
    model = ARIMA(train, order=(5,2,0))  # Modify order as appropriate for the dataset
    model_fit = model.fit()

    # Log model parameters
    mlflow.log_param("ARIMA order", "(5,2,0)")

    # Make predictions
    predictions = model_fit.forecast(steps=len(test))

    # Calculate and log mean squared error
    mse = mean_squared_error(test, predictions)
    mlflow.log_metric("MSE", mse)

    # Save the model to file
    model_fit.save("model_arima.pkl")
    mlflow.log_artifact("model_arima.pkl", "model")

    # Log the sklearn model and register as version 1
    mlflow.statsmodels.log_model(
        statsmodels_model=model_fit,
        artifact_path="stats-model",
        registered_model_name="arima_model",
    )

Writing app.py


In [ ]:
34.73.144.201

In [6]:
#################
# RUN MLFFLOW UI
#################
import subprocess

# Start app.py in the background
subprocess.Popen(["python", "app.py"])

# Start MLflow UI in the background
subprocess.Popen(["mlflow", "ui"])

# # Using subprocess to run localtunnel and capture output
# lt_process = subprocess.Popen(["npx", "localtunnel", "--port", "5000"], stdout=subprocess.PIPE, text=True)

# # Print the output of localtunnel (e.g., the URL)
# while True:
#     line = lt_process.stdout.readline()
#     if not line:
#         break
#     print(line.strip())


<Popen: returncode: None args: ['mlflow', 'ui']>

In [40]:
#################
# BUILD DOCKER CONTAINER
#################
!mlflow models build-docker --model-uri "models:/arima_model/1" --name "arima_model_mlops"


2024/04/21 21:35:25 INFO mlflow.models.flavor_backend_registry: Selected backend for flavor 'python_function'
2024/04/21 21:35:25 INFO mlflow.pyfunc.backend: Building docker image with name arima_model_mlops
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py", line 791, in urlopen
    response = self._make_request(
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py", line 497, in _make_request
    conn.request(
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connection.py", line 395, in request
    self.endheaders()
  File "/usr/lib/python3.10/http/client.py", line 1278, in endheaders
    self._send_output(message_body, encode_chunked=encode_chunked)
  File "/usr/lib/python3.10/http/client.py", line 1038, in _send_output
    self.send(msg)
  File "/usr/lib/python3.10/http/client.py", line 976, in send
    self.connect()
  File "/usr/local/lib/python3.10/dist-packages/docker/transport/unixconn.py", 